In [13]:
# import dependencies
import pandas as pd
import pymongo

In [153]:
# define csv to import
csv = 'game_database_final.csv'

In [167]:
# read csv and preview
df = pd.read_csv(csv)
df

,Game,Month,Hours Watched,Average Viewers,Average Concurrent Streams,Peak Concurrent Streams,Peak Viewers
0,Among Us,1/1/21,27.1M,42.0K,895,2.4K,269K
1,Among Us,12/1/20,50.3M,67.7K,1.2K,3.3K,499K
2,Among Us,11/1/20,77.0M,107K,1.8K,5.3K,774K
3,Among Us,10/1/20,83.4M,3.0K,151K,754K,9.1K
4,Among Us,9/1/20,147M,3.5K,204K,743K,12.4K
...,...,...,...,...,...,...,...
186,League of Legends,4/1/17,82.5M,115K,1.6K,2.5K,488K
187,League of Legends,3/1/17,90.8M,122K,1.6K,2.5K,326K
188,League of Legends,2/1/17,99.3M,148K,1.8K,2.8K,391K
189,League of Legends,1/1/17,99.1M,134K,1.8K,2.9K,378K


In [168]:
# rename columns
columns = ['game', 'month', 'hours_watched', 'average_viewers(k)', 'average_concurrent_streams(k)', 'peak_concurrent_streams(k)', 'peak_viewers(k)']
df.columns = columns
df

,game,month,hours_watched,average_viewers(k),average_concurrent_streams(k),peak_concurrent_streams(k),peak_viewers(k)
0,Among Us,1/1/21,27.1M,42.0K,895,2.4K,269K
1,Among Us,12/1/20,50.3M,67.7K,1.2K,3.3K,499K
2,Among Us,11/1/20,77.0M,107K,1.8K,5.3K,774K
3,Among Us,10/1/20,83.4M,3.0K,151K,754K,9.1K
4,Among Us,9/1/20,147M,3.5K,204K,743K,12.4K
...,...,...,...,...,...,...,...
186,League of Legends,4/1/17,82.5M,115K,1.6K,2.5K,488K
187,League of Legends,3/1/17,90.8M,122K,1.6K,2.5K,326K
188,League of Legends,2/1/17,99.3M,148K,1.8K,2.8K,391K
189,League of Legends,1/1/17,99.1M,134K,1.8K,2.9K,378K


In [169]:
# format columns
df['month'] = pd.to_datetime(df['month'], format='%d/%m/%y')
df['hours_watched'] = df['hours_watched'].map(lambda x: float(x.strip('M')) if 'M' in x else float(x.strip('K'))/1000000)
df['average_viewers(k)'] = df['average_viewers(k)'].map(lambda x: float(x.strip('K')))
df['average_concurrent_streams(k)'] = df['average_concurrent_streams(k)'].map(lambda x: float(x.strip('K')) if 'K' in x else float(x)/1000)
df['peak_concurrent_streams(k)'] = df['peak_concurrent_streams(k)'].map(lambda x: float(x.strip('K')) if 'K' in x else float(x)/1000)
df['peak_viewers(k)'] = df['peak_viewers(k)'].map(lambda x: float(x.strip('K')) if 'K' in x else float(x.strip('M'))*1000)
df

,game,month,hours_watched,average_viewers(k),average_concurrent_streams(k),peak_concurrent_streams(k),peak_viewers(k)
0,Among Us,2021-01-01,27.1,42.0,0.895,2.4,269.0
1,Among Us,2020-01-12,50.3,67.7,1.200,3.3,499.0
2,Among Us,2020-01-11,77.0,107.0,1.800,5.3,774.0
3,Among Us,2020-01-10,83.4,3.0,151.000,754.0,9.1
4,Among Us,2020-01-09,147.0,3.5,204.000,743.0,12.4
...,...,...,...,...,...,...,...
186,League of Legends,2017-01-04,82.5,115.0,1.600,2.5,488.0
187,League of Legends,2017-01-03,90.8,122.0,1.600,2.5,326.0
188,League of Legends,2017-01-02,99.3,148.0,1.800,2.8,391.0
189,League of Legends,2017-01-01,99.1,134.0,1.800,2.9,378.0


In [170]:
# check names for unique games in dataframe
df['game'].unique()

array(['Among Us', 'Call of Duty ', 'Counter Strike Global Offensive',
       'Fortnite', 'League of Legends '], dtype=object)

In [171]:
# split data for each game into separate dataframes
amongus_df = df.loc[df['game'] == 'Among Us']
cod_df = df.loc[df['game'] == 'Call of Duty ']
csgo_df = df.loc[df['game'] == 'Counter Strike Global Offensive']
fortnite_df = df.loc[df['game'] == 'Fortnite']
lol_df = df.loc[df['game'] == 'League of Legends ']

In [172]:
# reset index and set month as index
df_list = [amongus_df, cod_df, csgo_df, fortnite_df, lol_df]
for x in df_list:
    x.reset_index(drop=True, inplace=True)
    x.set_index('month', inplace=True)

In [173]:
# preview df sample
lol_df

,game,hours_watched,average_viewers(k),average_concurrent_streams(k),peak_concurrent_streams(k),peak_viewers(k)
month,,,,,,
2021-01-01,League of Legends,145.0,224.0,5.8,11.6,699.0
2020-01-12,League of Legends,96.2,130.0,4.4,8.0,320.0
2020-01-11,League of Legends,121.0,168.0,4.7,8.9,345.0
2020-01-10,League of Legends,125.0,226.0,4.5,8.5,1420.0
2020-01-09,League of Legends,135.0,187.0,4.4,6.8,795.0
2020-01-08,League of Legends,142.0,192.0,4.4,7.5,730.0
2020-01-07,League of Legends,135.0,182.0,4.4,7.3,505.0
2020-01-06,League of Legends,143.0,199.0,4.2,7.5,561.0
2020-01-05,League of Legends,127.0,172.0,4.7,8.3,507.0


In [18]:
def setup_game_db():
    
    # define csv to import
    csv = 'game_database_final.csv'
    
    # read csv and preview
    df = pd.read_csv(csv)
    
    # rename columns
    columns = ['game', 'month', 'hours_watched', 'average_viewers(k)', 'average_concurrent_streams(k)', 'peak_concurrent_streams(k)', 'peak_viewers(k)']
    df.columns = columns
    
    # format columns
#     df['month'] = pd.to_datetime(df['month'], format='%d/%m/%y')
    df['hours_watched'] = df['hours_watched'].map(lambda x: float(x.strip('M')) if 'M' in x else float(x.strip('K'))/1000000)
    df['average_viewers(k)'] = df['average_viewers(k)'].map(lambda x: float(x.strip('K')))
    df['average_concurrent_streams(k)'] = df['average_concurrent_streams(k)'].map(lambda x: float(x.strip('K')) if 'K' in x else float(x)/1000)
    df['peak_concurrent_streams(k)'] = df['peak_concurrent_streams(k)'].map(lambda x: float(x.strip('K')) if 'K' in x else float(x)/1000)
    df['peak_viewers(k)'] = df['peak_viewers(k)'].map(lambda x: float(x.strip('K')) if 'K' in x else float(x.strip('M'))*1000)

    # split data for each game into separate dataframes
    amongus_df = df.loc[df['game'] == 'Among Us']
    cod_df = df.loc[df['game'] == 'Call of Duty ']
    csgo_df = df.loc[df['game'] == 'Counter Strike Global Offensive']
    fortnite_df = df.loc[df['game'] == 'Fortnite']
    lol_df = df.loc[df['game'] == 'League of Legends ']
    
    # reset index and set month as index
    df_list = [amongus_df, cod_df, csgo_df, fortnite_df, lol_df]
    for x in df_list:
        x.reset_index(drop=True, inplace=True)
        x.set_index('month', inplace=True)
        
    return df_list

In [21]:
def df_rows_todict(dataframe):
    df_dict = {}
    for index, row in dataframe.iterrows():
        column_values = row.to_dict()
        df_dict[index] = column_values
    return df_dict

In [22]:
def upload_db(collection, dictionary):
    db_name = 'project2'
    myclient = pymongo.MongoClient('mongodb://localhost:27017/')
    mydb = myclient[db_name]
    mycol = mydb[collection]
    # error handling in case collection already exists in database
    try:
        upload = mycol.insert_one(dictionary)
    except:
        print('This collection already exists. Change collection name or delete collection in mongodb before trying again.')

In [24]:
df_list = setup_game_db()

In [31]:
db_names = ['amongus_db', 'cod_db', 'csgo_db', 'fortnite_db', 'lol_db']
for index, value in enumerate(df_list):
    collection = db_names[index]
    df = df_list[index]
    dictionary = df_rows_todict(df)
    upload_db(collection, dictionary)